In [1]:
%load_ext rpy2.ipython

In [2]:
%R require(gender)
%R require(genderdata)
%R require(tibble)

R[write to console]: Loading required package: gender

R[write to console]: Loading required package: genderdata

R[write to console]: Loading required package: tibble



1


In [3]:
%matplotlib notebook

import pandas as pd
import numpy as np

import time
import pickle

import memory_profiler
%load_ext memory_profiler

from pathlib import Path

In [4]:
import black
import jupyter_black

jupyter_black.load(line_length=79)

In [5]:
variables_path = Path("../results/variables/iclr24v2")
figures_path = Path("../results/figures")
data_path = Path("../data")

In [6]:
%watermark -a 'Rita González-Márquez' -t -d -tz -u -v -iv -w -m -h -p transformers,openTSNE
print(distro.name(pretty=True))

UsageError: Line magic function `%watermark` not found.


# Import

In [7]:
%%time
iclr2024 = pd.read_parquet(
    data_path / "iclr24v2.parquet",
    engine="pyarrow",
)

CPU times: user 318 ms, sys: 128 ms, total: 446 ms
Wall time: 1.17 s


In [8]:
iclr2024.keywords = iclr2024.keywords.transform(lambda x: list(x))
iclr2024.scores = iclr2024.scores.transform(lambda x: list(x))

In [9]:
iclr2024

,year,id,title,abstract,authors,decision,scores,keywords,labels
0,2017,B1-Hhnslg,Prototypical Networks for Few-shot Learning,A recent approach to few-shot classification c...,"Jake Snell, Kevin Swersky, Richard Zemel",Reject,"[6, 4, 5]","[deep learning, transfer learning]",transfer learning
1,2017,B1-q5Pqxl,Machine Comprehension Using Match-LSTM and Ans...,Machine comprehension of text is an important ...,"Shuohang Wang, Jing Jiang",Accept (Poster),"[6, 6, 7]","[natural language processing, deep learning]",language models
2,2017,B16Jem9xe,Learning in Implicit Generative Models,Generative adversarial networks (GANs) provide...,"Shakir Mohamed, Balaji Lakshminarayanan",Invite to Workshop Track,"[8, 7, 6]",[unsupervised learning],unlabeled
3,2017,B16dGcqlx,Third Person Imitation Learning,Reinforcement learning (RL) makes it possible ...,"Bradly C Stadie, Pieter Abbeel, Ilya Sutskever",Accept (Poster),"[6, 5, 6]",[],unlabeled
4,2017,B184E5qee,Improving Neural Language Models with a Contin...,We propose an extension to neural network lang...,"Edouard Grave, Armand Joulin, Nicolas Usunier",Accept (Poster),"[7, 9, 5]",[natural language processing],language models
...,...,...,...,...,...,...,...,...,...
24343,2024,zxPDdw8koz,CLIP meets Model Zoo Experts: Pseudo-Supervisi...,Contrastive language image pretraining (CLIP) ...,,Withdrawn,"[8, 3, 3, 3]","[contrastive learning, clip, distillation, den...",vision-language models
24344,2024,zyBJodMrn5,On the generalization capacity of neural netwo...,The advent of the Transformer has led to the d...,"Takuya Ito, Soham Dan, Mattia Rigotti, James K...",Accept (poster),"[8, 3, 6]","[compositional generalization, compositionalit...",out-of-distribution
24345,2024,zz61V8bIab,Stochastic Adversarial Networks for Multi-Doma...,Adversarial training has played a pivotal role...,,Withdrawn,"[5, 1, 5]","[multi-domain text classification, adversarial...",adversarial
24346,2024,zzqn5G9fjn,Breaking Physical and Linguistic Borders: Mult...,Pretrained large language models (LLMs) have e...,"Wanru Zhao, Royson Lee, Yihong Chen, Xinchi Qi...",Accept (poster),"[5, 3, 1, 8]","[multilingual federated learning, natural lang...",language models


## Split names

In [36]:
authors = iclr2024.authors.to_list()
authors[:10]

['Jake Snell, Kevin Swersky, Richard Zemel',
 'Shuohang Wang, Jing Jiang',
 'Shakir Mohamed, Balaji Lakshminarayanan',
 'Bradly C Stadie, Pieter Abbeel, Ilya Sutskever',
 'Edouard Grave, Armand Joulin, Nicolas Usunier',
 'Levent Sagun, Leon Bottou, Yann LeCun',
 'Jianwen Xie, Yang Lu, Ruiqi Gao, Song-Chun Zhu, Ying Nian Wu',
 'Vincent Dumoulin, Ishmael Belghazi, Ben Poole, Alex Lamb, Martin Arjovsky, Olivier Mastropietro, Aaron Courville',
 'Joji Toyama, Masanori Misono, Masahiro Suzuki, Kotaro Nakayama, Yutaka Matsuo',
 'Sahil Sharma, Aravind S. Lakshminarayanan, Balaraman Ravindran']

In [38]:
%%time
first_author_first_name = [
    elem.split(",")[0].strip().split(" ")[0] for elem in authors
]

CPU times: user 23.2 ms, sys: 2.49 ms, total: 25.7 ms
Wall time: 24.6 ms


In [39]:
%%time
last_author_first_name = [
    elem.split(",")[-1].strip().split(" ")[0] for elem in authors
]

CPU times: user 23.6 ms, sys: 1.48 ms, total: 25.1 ms
Wall time: 24.4 ms


### Exploration

In [83]:
print(np.sum(np.array(first_author_first_name) == ""))
print(np.sum(np.array(last_author_first_name) == ""))

1553
1553


# Gender prediction

##  First author

In [45]:
dict_names = {"Names": first_author_first_name}
df = pd.DataFrame(dict_names)
df_names_first_author = df.Names
df_names_first_author

0            Jake
1        Shuohang
2          Shakir
3          Bradly
4         Edouard
           ...   
24343            
24344      Takuya
24345            
24346       Wanru
24347            
Name: Names, Length: 24348, dtype: object

In [47]:
%%time
%R library(gender)
%R -i df_names_first_author -o result result = gender(df_names_first_author, years = 2012, method = "ssa")

CPU times: user 1.16 s, sys: 30.2 ms, total: 1.19 s
Wall time: 1.19 s


,name,proportion_male,proportion_female,gender,year_min,year_max
1,Aadarsh,1.0000,0.0000,male,2012.0,2012.0
2,Aakash,1.0000,0.0000,male,2012.0,2012.0
3,Aaron,0.9970,0.0030,male,2012.0,2012.0
4,Aaron,0.9970,0.0030,male,2012.0,2012.0
5,Aaron,0.9970,0.0030,male,2012.0,2012.0
...,...,...,...,...,...,...
9695,Zoe,0.0023,0.9977,female,2012.0,2012.0
9696,Zoe,0.0023,0.9977,female,2012.0,2012.0
9697,Zohar,0.5149,0.4851,male,2012.0,2012.0
9698,Zohar,0.5149,0.4851,male,2012.0,2012.0


In [49]:
gender_map = dict(zip(result.name, result.gender))

In [50]:
first_author_gender = np.vectorize(gender_map.get)(first_author_first_name)

##  Last author

In [54]:
dict_names = {"Names": last_author_first_name}
df = pd.DataFrame(dict_names)
df_names_last_author = df.Names
df_names_last_author

0         Richard
1            Jing
2          Balaji
3            Ilya
4         Nicolas
           ...   
24343            
24344      Murray
24345            
24346    Nicholas
24347            
Name: Names, Length: 24348, dtype: object

In [55]:
%%time
%R library(gender)
%R -i df_names_last_author -o result result = gender(df_names_last_author, years = 2012, method = "ssa")

CPU times: user 575 ms, sys: 6.23 ms, total: 582 ms
Wall time: 581 ms


,name,proportion_male,proportion_female,gender,year_min,year_max
1,Aaditya,1.0000,0.0000,male,2012.0,2012.0
2,Aaditya,1.0000,0.0000,male,2012.0,2012.0
3,Aaro,1.0000,0.0000,male,2012.0,2012.0
4,Aaron,0.9970,0.0030,male,2012.0,2012.0
5,Aaron,0.9970,0.0030,male,2012.0,2012.0
...,...,...,...,...,...,...
11540,Zohar,0.5149,0.4851,male,2012.0,2012.0
11541,Zohar,0.5149,0.4851,male,2012.0,2012.0
11542,Zoran,1.0000,0.0000,male,2012.0,2012.0
11543,Zoran,1.0000,0.0000,male,2012.0,2012.0


In [56]:
gender_map = dict(zip(result.name, result.gender))

In [57]:
last_author_gender = np.vectorize(gender_map.get)(last_author_first_name)

In [59]:
last_author_gender

array(['male', 'female', 'None', ..., 'None', 'male', 'None'], dtype='<U6')

## Exploration

In [64]:
print(np.sum(first_author_gender == "male"))
print(np.sum(first_author_gender == "female"))
print(np.sum(first_author_gender == "None"))

8042
1657
14649


In [62]:
print(
    np.sum(first_author_gender == "male")
    + np.sum(first_author_gender == "female")
    + np.sum(first_author_gender == "None")
)
first_author_gender.shape

24348


(24348,)

In [63]:
print(np.sum(last_author_gender == "male"))
print(np.sum(last_author_gender == "female"))
print(np.sum(last_author_gender == "None"))

9889
1655
12804


In [65]:
print(
    np.sum(last_author_gender == "male")
    + np.sum(last_author_gender == "female")
    + np.sum(last_author_gender == "None")
)

24348


# Save

In [68]:
dict_names = {
    "first_author_name": first_author_first_name,
    "first_author_gender": first_author_gender,
    "last_author_name": last_author_first_name,
    "last_author_gender": last_author_gender,
}
df = pd.DataFrame(dict_names)
df

,first_author_name,first_author_gender,last_author_name,last_author_gender
0,Jake,male,Richard,male
1,Shuohang,None,Jing,female
2,Shakir,male,Balaji,None
3,Bradly,male,Ilya,male
4,Edouard,male,Nicolas,male
...,...,...,...,...
24343,,None,,None
24344,Takuya,None,Murray,male
24345,,None,,None
24346,Wanru,None,Nicholas,male


In [69]:
# save
df.to_parquet(
    variables_path / "names_and_genders.parquet",
    index=False,
    engine="pyarrow",
)